# pySAS Introduction -- Short Version
---
This tutorial provides a short, basic introduction to using pySAS on your computer. It only covers how to download observation data files and how to calibrate the data.  A much more comprehensive introduction can be found in the <a href="./xmm-pysas-intro-long.ipynb">Long pySAS Introduction</a>. This tutorial is intened for those who are already familiar with SAS commands and want to use Python to run SAS commands. A tutorial on how to learn to use SAS and pySAS for XMM analysis can be found in <a href="./xmm_ABC_guide_images_and_filters.ipynb">The XMM-Newton ABC Guide</a>. In this tutorial we will demonstrate,

1. How to select a directory for data and analysis.
2. How to copy XMM data from the HEASARC archive.
3. How to run the standard XMM SAS commands `cfibuild` and `odfingest`.

This tutorial uses an observation of NGC3079 (`obsid = '0802710101'`).

<div class="alert alert-block alert-info">
    <b>Note:</b> Before running this notebook, or even starting a Jupyter Lab session, HEASOFT has to be initialized. If you did not initalize HEASOFT before starting this Jupyter Lab session, or opening this notebook, please close this window and initalize HEASOFT (it is not possible to initalize HEASOFT from within a Jupyter Notebook). SAS defaults will need to be set as explained in the README on GitHub (https://github.com/XMMGOF/pysas/blob/main/README.md).
</div>

- [XMM Newton GOF Helpdesk](https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html "Helpdesk") - Link to form to contact the GOF Helpdesk.
- [`pysas` on GitHub](https://github.com/XMMGOF/pysas)

## Last Reviewed: _29 July 2024, for SAS v21_
##### Last Updated: _29 July 2024_
##### By: Ryan Tanner (ryan.tanner@nasa.gov)
---

In [ ]:
import os
import pysas
obsid = '0802710101'

## Run `odf.calibrate_odf`

When you run the cell below the following things will happen.

1. `calibrate_odf` will check if `data_dir` exists, and if not it will create it.
2. Inside data_dir `calibrate_odf` will create a directory with the value for the obs ID (i.e. `$data_dir/0802710101/`).
3. Inside of that, `calibrate_odf` will create two directories:

    a. `$data_dir/0802710101/ODF` where the observation data files are kept.
    
    b. `$data_dir/0802710101/work` where the `ccf.cif`, `*SUM.SAS`, and output files are kept. All SAS tasks should be run from this directory.
4. `calibrate_odf` will automatically transfer the data for `obsid` to `$data_dir/0802710101/ODF` from the HEASARC archive.
5. `calibrate_odf` will run `cfibuild` and `odfingest`.

That is it! Your data is now calibrated and ready for use with all the standard SAS commands!

In [ ]:
odf = pysas.odfcontrol.ODFobject(obsid)
odf.calibrate_odf(repo='heasarc',overwrite=True)

If you want more information on the function `calibrate_odf` run the cell below or see the long introduction tutorial.

In [ ]:
odf.calibrate_odf?

---
To run SAS tasks, especially ones not written in Python, you will need to import a wrapper from pySAS. SAS tasks should be run from the work directory. The location of the work direcotry is stored as a variable in `odf.work_dir`.

In [ ]:
from pysas.wrapper import Wrapper as w
os.chdir(odf.work_dir)

The wrapper, imported as `w`, takes two inputs, the name of the SAS task to run, and a Python list of all the input arguments for that task. For example, to run a task with no input arguments you simply provide an empty list as the second argument.

In [ ]:
inargs = []
w('epproc', inargs).run()

In [ ]:
w('emproc', []).run()

In [ ]:
w('rgsproc', []).run()

You can list all input arguments available to any SAS task with option `'--help'` (or `'-h'`),

In [ ]:
w('epproc', ['-h']).run()

If there are multiple input arguments then each needs to be a separate string in the Python list.

Here is an example of how to apply a "standard" filter. This is equivelant to running the following SAS command:

```
evselect table=unfiltered_event_list.fits withfilteredset=yes \
    expression='(PATTERN $<=$ 12)&&(PI in [200:12000])&&#XMMEA_EM' \
    filteredset=filtered_event_list.fits filtertype=expression keepfilteroutput=yes \
    updateexposure=yes filterexposure=yes
```

The input arguments should be in a list, with each input argument a separate string. Note: Some inputs require single quotes to be preserved in the string. This can be done using double quotes to form the string. i.e. `"expression='(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'"`

In [ ]:
unfiltered_event_list = odf.files['M1evt_list'][0]

inargs = ['table={0}'.format(unfiltered_event_list), 
          'withfilteredset=yes', 
          "expression='(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'", 
          'filteredset=filtered_event_list.fits', 
          'filtertype=expression', 
          'keepfilteroutput=yes', 
          'updateexposure=yes', 
          'filterexposure=yes']

w('evselect', inargs).run()